In [1]:
#!/usr/bin/env python                                                                                                                                             
# -*- coding:utf-8 -*-  
import json
from requests_oauthlib import OAuth1Session
from twitter import Twitter, OAuth
from janome.tokenizer import Tokenizer
import collections
import re
from collections import Counter, defaultdict
import sys, json, time, calendar
from datetime import datetime
from dateutil.relativedelta import relativedelta

#APIキーの設置
CONSUMER_KEY =  'Ojj0Pxe8CQ0ClLvnP1OQnvbBT'
CONSUMER_SECRET = 'UiAFp6A9AvSb3Q7tD3NY5nr8bmGIvCtnGyQzlhnHxHeyY5bJI8'
ACCESS_TOKEN = '999656799220322304-YJRSS4O7VvsZRCt2rsjrD32uiDs1rIC'
ACCESS_SECRET = 'BNEd1qWjZzuwdiwRUl2xglkqpCrnR9dwgcdJLQfkUuGsk'

t = Twitter(auth=OAuth(
    ACCESS_TOKEN,
    ACCESS_SECRET,
    CONSUMER_KEY,
    CONSUMER_SECRET
))
    
twitter = OAuth1Session(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
url = "https://api.twitter.com/1.1/search/tweets.json"
userTweets = []

def get_userstweets_again(screen_name, max_id):
    max_id = max_id
    count = 200 #一度のアクセスで何件取ってくるか
    aTimeLine = t.statuses.user_timeline(screen_name = screen_name, count=count, max_id=max_id)
    for tweet in aTimeLine:
        userTweets.append(tweet['text'])

def get_userstweets(screen_name):
    number_of_tweets = 0
    count = 200 #一度のアクセスで何件取ってくるか
    aTimeLine = t.statuses.user_timeline(screen_name = screen_name, count=count)
    for tweet in aTimeLine:
        number_of_tweets += 1
        userTweets.append(tweet['text'])
        if number_of_tweets >= 200:
            max_id = tweet["id"]
            print(max_id)
            get_userstweets_again(screen_name, max_id)
    return userTweets
            
            
def get_shaped_tweets(tweets_list):
    shaped_tweets = []
    rm_replie = re.compile(r'@([A-Za-z0-9_]+)')
    rm_url = re.compile(r'https?://t.co/([A-Za-z0-9_]+)')
    rm_hashtag = re.compile(r'#(\w+)')
    for tweet in tweets_list:
        shape = rm_replie.sub('', tweet)
        shape = rm_url.sub('', shape)
        shape = rm_hashtag.sub('', shape)
        shape = shape.replace('&gt;', '>').replace('&lt;', '<').replace('&amp;', '&').replace('\n', ' ')
        shaped_tweets.append(shape)
    return shaped_tweets



#検索したい相手を指定 
print("自分のuser_idを入力してください")
my_user_id = input('>> ')
print('----------------------------------------------------')

tweets = get_userstweets(my_user_id)
tweets = get_shaped_tweets(tweets)


print(tweets)
print(len(tweets))


自分のuser_idを入力してください
>> ks_mocchaso
----------------------------------------------------
['scipyのsparse強すぎて感動してる', 'RT : 【応募用ツイート】 Progate登録ユーザー数が50万人を突破しました\U0001f929😭みなさん本当にありがとうございます。感謝を込めて「パーカー＆Tシャツが当たるTwitterキャンペーン」を実施します❤️応募は をフォローしRTするだけ！詳しくはこ…', '5〜6分経てばフリーズは直るけど、In [*]のままだ…', 'あー、特徴ベクトルファイルに保存して読み込んでみたらフリーズした  結局フリーズ問題は付いてくるのか…', '特徴ベクトルづくりだけでこんなに時間食うとは… このままだとクソみたいな発表しかできんぞ…', '19:00寝3:30起きをキメた(晩飯前に寝てたら寝過ぎただけ)', ' GPUほぼ必須ですもんね！ 学生には辛い…。', ' 意外とあるあるなんですね！ レアなケースを引き当てたのかと思いましたw', '最低限のノルマまでは達成できそうなのにコーディング進まん笑 先に抄録とスライドづくりに着手するか…', 'ログの出し過ぎでjupyter固まってPCまでフリーズしたから、ファイル分割してtqdmまで導入したのに、まだ止まる時ある😭', '21893粒のポップコーンを召喚しました！【 伝説のポップコーンマスター 】  ', '全40問をクリアしました！ 所要時間は【47.2秒】です。  ', 'はてなブログに投稿しました  Git LFSでpushできるようになった！ - mocchaso - もっちゃそ ', 'Twitter APIの登録思ったよりだるくて草', ' 何となく検索かけてみたら、出てきてビックリしました ', 'すげえ、TrelloってiOSアプリ版もあるのか…！', 'Git LFS導入できぬ', 'Windows8、MeCabは何とか入れられるけどNEOlogdインストールできねえんか 今後の課題に挙げようかな…笑', '卒研関係で貰ったデータに不備があってキレそう', 'Slack連携のテスト〜', 'RT : プログラミング学ぶ上で、1番大事なのが逆算です。  ・何ができるよ

# カテゴリの決定(本でやったのと同じ)

In [2]:
#抽出したユーザーのツイートから性格を出す

In [3]:
import json
from watson_developer_cloud import PersonalityInsightsV3
from decimal import Decimal, ROUND_HALF_UP, ROUND_HALF_EVEN
from collections import OrderedDict

def get_personality(text):
    """
    Personality Insightsを用いた性格分析の結果を取得する
    参考サイト：http://tmngtm.hatenablog.com/entry/2016/10/14/203901
    -> return responseで400コードが出てうまくいかず(後で分かったことだが、恐らくContent-Languageをenにしていた...)
    -> 公式マニュアルを読み解いた
    ※事前に、pip install --upgrade watson-developer-cloud
    """
    api_version = "2017-10-13"
    api_username = "396677bb-f0d3-4650-9d16-9e479f9e1f78"
    api_password = "CyJVefXvDdIA"
    api_url = "https://gateway.watsonplatform.net/personality-insights/api"
    
    personality_insights = PersonalityInsightsV3(
        version = api_version,
        username = api_username,
        password = api_password,
        url = api_url
    )
    
    profile = personality_insights.profile(
        content = text,
        content_type = "text/plain",
        accept = "application/json",
        content_language = "ja",
        accept_language = "en",
        raw_scores = True,
    )
    
    return profile

def convert_dict_to_json(orig_dict, indent = 4):
    """
    辞書からJSON形式に変換する
    参考サイト：https://tmg0525.hatenadiary.jp/entry/2018/03/30/204448
    """
    return json.dumps(orig_dict)

def load_json_as_dict(json_name):
    """
    JSON形式の文字列を辞書として読み込む
    参考サイト：https://note.nkmk.me/python-json-load-dump/
    """
    with open("./" + json_name, "r") as json_file:
        return json.load(json_file, object_pairs_hook = OrderedDict)

def sort_personality(tweet_result):
    """
    tweet_resultを性格の値で降順にソートし、リストとしてreturnする
    returnされたリストをスライスすれば一部を取得できる
    カテゴリ分けの時に使う
    """
    return sorted(tweet_result.items(), key = lambda x: x[1], reverse = True)

In [4]:
## ユーザのツイートを性格APIで分析する

tweets_joined = " ".join(tweets) # 集めた複数のツイートを結合
user_personality_dict = OrderedDict()

tweet_personality_dict = get_personality(tweets_joined) # ツイートを結合したデータをAPIに入力
# ツイートから分析した性格の名前
personality_name = [big5["name"] for big5 in tweet_personality_dict["personality"]]
# 性格の値。小数点第2位で四捨五入した後、decimal.Decimalからfloatに変換している。
# 参考サイト(四捨五入)：https://note.nkmk.me/python-round-decimal-quantize/
personality_percentile = [float(Decimal(str(big5["percentile"])).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)) \
                          for big5 in tweet_personality_dict["personality"]]

# ユーザのツイートの分析結果
all_tweets_big5 = OrderedDict(zip(personality_name, personality_percentile))

# { my_user_id: { personality_name1: percentile1, personality2: percentile2, ... } }
user_personality_dict[my_user_id] = all_tweets_big5
print("finished analyzing all({}) tweets.".format(len(tweets)))

finished analyzing all(76) tweets.


In [13]:
## 分析結果を基に、本のカテゴリ分けを行う。

import pprint
import itertools

# Agreeableness: 協調性, Conscientiousness: 真面目さ, Emotional range: 精神的安定性, Extraversion: 外向性, Openness: 開放性
# 参考サイト：https://note.nkmk.me/python-math-factorial-permutations-combinations/
big5_list = ["Agreeableness", "Conscientiousness", "Emotional range", "Extraversion", "Openness"]

# big5から2つの性格を選ぶ組み合わせを列挙する
category_patterns = [set(pat) for pat in itertools.combinations(big5_list, 2)]

# ユーザのツイートのカテゴリ分けを記録する辞書
user_tweet_category_table = OrderedDict()

# ユーザのツイートの分析結果を、値の大きい順にソートする
sorted_result = sort_personality(user_personality_dict[my_user_id])

# big5のうち大きい順から2番目までの値の性格名のみを取得し、その本のカテゴリとする(例：{Openness, Extraversion})
# この時、複数の性格の順番を考慮させないために、setで用意する
# 2つを組み合わせているため、1つだけの場合よりも幅広くカテゴリ分けできそう
tweet_category = {big5_elm[0] for big5_elm in sorted_result[0:2]}
user_tweet_category_table[my_user_id] = tweet_category

print("category_patterns = ")
pprint.pprint(category_patterns)
print()
pprint.pprint(user_tweet_category_table)

category_patterns = 
[{'Conscientiousness', 'Agreeableness'},
 {'Emotional range', 'Agreeableness'},
 {'Extraversion', 'Agreeableness'},
 {'Openness', 'Agreeableness'},
 {'Emotional range', 'Conscientiousness'},
 {'Extraversion', 'Conscientiousness'},
 {'Openness', 'Conscientiousness'},
 {'Emotional range', 'Extraversion'},
 {'Emotional range', 'Openness'},
 {'Openness', 'Extraversion'}]

OrderedDict([('ks_mocchaso', {'Emotional range', 'Conscientiousness'})])


# 条件分岐

In [14]:
#本のカテゴリと一致した場合その本をサジェストする。

In [15]:
#こんな感じ
import random

user_personality = user_tweet_category_table[my_user_id]

suggest_book = ""
if user_personality == {'Openness', 'Extraversion'}:
    candidates = ['chumonno_oi_ryoriten', 'hashire_merosu']
    suggest_book = random.choice(candidates)
    print("book: {} was suggested.".format(suggest_book))
elif user_personality == {'Openness', 'Emotional range'}: 
    suggest_book = 'bocchan'
    print("book: {} was suggested.".format(suggest_book))
elif user_personality == {'Openness', 'Agreeableness'}:
    candidates = ['kaijin_nijumenso', 'ningen_shikkaku']
    suggest_book = random.choice(candidates)
    print("book: {} was suggested.".format(suggest_book))
else:
    print("not found a sugested book...")

not found a sugested book...
